# Database Querying with a Uniform Syntax

In my academic days, before Google Scholar came along, I was a big user of [PubMed](https://pubmed.ncbi.nlm.nih.gov/) for searching through the scientific literature. I became very familiar with PubMed's query syntax - things of the sort:

``` parsons sp[au] and (intestine[title] or channel[title]) ```

When I was writing a search engine at my first industry job, I thought this would make a great interface. The UI would only need a single text box for inputting the query. I had seen too many times in UIs the following pattern. Each field is queried with a separate UI element(s), e.g. a text box for each string field, a check box for each boolean field, a text box and drop-down comparator for each numeric field. The fields can only be combined in limited ways (all OR or all AND) or else there has to be a bunch more UI elements for selecting AND, OR, NOT. Boolean expression can certainly not be combined in parentheses.

Implementing a simple PubMed-style query, requires a parser to parse the query. I came across the [Pyparsing package](https://pyparsing-docs.readthedocs.io/en/latest/index.html) which was included in some other package I was using (I can't remember which) and from there the booklet, "Getting started with Pyparsing" by Paul McGuire (2007, O'Reilly). A section of that book ("Parsing a search string") essentially gave me the code I was looking for. Perfect!

In this repository I have written an class (`Query`) that does the parsing outlined in McGuire. This is a base (abtract) class. It can be inherited from to implement the same PubMed-like query interface for different kinds of database. `ObjectListQuery(Query)` is used to query a list of objects of any type (class objects, dictionaries, etc.). Fields are added simply with a one-line specification. `SQLQuery(Query)` is used to generate SQL queries. You can write your own subclass!


#### References

McGuire (2007). Getting started with Pyparsing. O'Reilly.

In [1]:
import datetime
from dataclasses import dataclass
import random
import sys
import time
import os

import sqlite3

sys.path.append(os.path.abspath(".."))
from resume.query.objectlistquery import ObjectListQuery
from resume.query.sqlitequery import SQLiteQuery
from resume.query.mongoquery import MongoQuery

## Querying a list of objects

The `ObjectListQuery` class lets you query any list of objects. The objects can be of any type.

Say we have a list of dictionaries, each dictionary describing a specimen in a zoological museum.

In [2]:
museum_specimens = [
    {
        "name": "zebra", "caught": datetime.datetime(2010, 3, 15),
        "dimensions": {"height": 1.4, "length": 1.8}, 
        "appendages": {"legs": 4}
    },
    {
        "name": "monkey", "caught": datetime.datetime(2002, 4, 11),
        "dimensions": {"height": 1.2}, 
        "appendages": {"legs": 2, "arms": 2}
    },
    {
        "name": "duck", "caught": datetime.datetime(1987, 11, 3),
        "dimensions": {"height": 0.15, "length": 0.24}, 
        "appendages": {"legs": 2, "wings": 2}, 
        "abilities": {"flys": True}
    },
    {
        "name": "whale", "caught": datetime.datetime(1910, 1, 15),
        "dimensions": {"height": 2.1, "length": 5.6}, 
        "appendages": {"legs": 2}
    },
    {
        "name": "millipede", "caught": datetime.datetime(1950, 7, 21),
        "dimensions": {"height": 0.005, "length": 0.04}, 
        "appendages": {"legs": 1000}
    },
]

We initiate an instance of `ObjectListQuery`, specifying the attributes ('fields') we want to search by.

Each field is specified by,

`(<full name>, <abbreviation>, <type>) : <function that returns its value from an object>`

Either the field's full name or abbreviation can be used in the search query, case insensitively. The type is used for type checking and interpretation of search operands. The function is used to retreive values from the objects and to index them.

In [3]:
querier = ObjectListQuery(fields={
    ("name", "nm", str): (lambda x: x["name"]),
    ("caught", "cg", datetime.datetime): (lambda x: x["caught"]),
    ("height", "hg", float): (lambda x: x["dimensions"]["height"]),
    ("legs", "lg", int): (lambda x: x["appendages"]["legs"]),
    ("arms", "ar", int): (lambda x: x["appendages"]["arms"]),
    ("flys", "fy", bool): (lambda x: x["abilities"]["flys"]),
})

Now add the specimens to the querier.

In [4]:
querier.add_objects(museum_specimens)

Now make some queries.

Search terms must be formatted as:

- String fields: can include a wildcard asterisk.

- Boolean fields: true is indicated by 't' or 'T'. false is anything else.

- Numeric (int or float) and datetime fields: must start with either >, < or = comparators.

In [5]:
querier.query("(<2000-01-01[caught] or z*[nm]) AND <10[LEGS]")

[{'name': 'zebra',
  'caught': datetime.datetime(2010, 3, 15, 0, 0),
  'dimensions': {'height': 1.4, 'length': 1.8},
  'appendages': {'legs': 4}},
 {'name': 'duck',
  'caught': datetime.datetime(1987, 11, 3, 0, 0),
  'dimensions': {'height': 0.15, 'length': 0.24},
  'appendages': {'legs': 2, 'wings': 2},
  'abilities': {'flys': True}},
 {'name': 'whale',
  'caught': datetime.datetime(1910, 1, 15, 0, 0),
  'dimensions': {'height': 2.1, 'length': 5.6},
  'appendages': {'legs': 2}}]

In [6]:
querier.query(">1.2[HEIGHT] AND >2[legs]")

[{'name': 'zebra',
  'caught': datetime.datetime(2010, 3, 15, 0, 0),
  'dimensions': {'height': 1.4, 'length': 1.8},
  'appendages': {'legs': 4}}]

In [7]:
querier.query(">1.2[HEIGHT] AND NOT >2[legs]")

[{'name': 'whale',
  'caught': datetime.datetime(1910, 1, 15, 0, 0),
  'dimensions': {'height': 2.1, 'length': 5.6},
  'appendages': {'legs': 2}}]

### Objects of any type

The objects can be of any type - we only need to specify the appropriate function for retrieving a field's value from the object.

For example,

In [8]:
@dataclass
class Specimen:
    name: str
    caught: datetime.datetime
    height: float
    legs: int
    arms: int

specimens_as_class_objects = [
    Specimen(name="horse", caught=datetime.datetime(1897, 10, 10), height=1.4, legs=4, arms=0),
    Specimen(name="cat", caught=datetime.datetime(1967, 5, 21), height=0.4, legs=4, arms=0),
    Specimen(name="gorilla", caught=datetime.datetime(1978, 2, 10), height=0.8, legs=2, arms=2),
    Specimen(name="spider", caught=datetime.datetime(1942, 10, 1), height=0.03, legs=8, arms=0),
    Specimen(name="lobster", caught=datetime.datetime(2005, 6, 12), height=0.1, legs=6, arms=2),
]

querier = ObjectListQuery(fields={ 
    ("name", "nm", str): (lambda x: x.name),
    ("caught", "ct", datetime.datetime): (lambda x: x.caught),
    ("height", "ht", float): (lambda x: x.height),
    ("legs", "lg", int): (lambda x: x.legs),
    ("arms", "ar", int): (lambda x: x.arms),
})

querier.add_objects(specimens_as_class_objects)

querier.query(">2[legs] AND *er[name]")

[Specimen(name='spider', caught=datetime.datetime(1942, 10, 1, 0, 0), height=0.03, legs=8, arms=0),
 Specimen(name='lobster', caught=datetime.datetime(2005, 6, 12, 0, 0), height=0.1, legs=6, arms=2)]

### Provide your own object identifiers

When you add objects with a call to `ObjectListQuery.add_objects()` it creates ["covering indexes"](https://en.wikipedia.org/wiki/Database_index). i.e. it associates every object with a unique ID and extracts the value for each field from the object and maps it ("indexes it") with its object's ID.

Normally `ObjectListQuery.add_objects()` will generate its own object IDs (a [UUID4](https://en.wikipedia.org/wiki/Universally_unique_identifier)). However you may already provide your objects with unique identifiers. In this case, you may provide a function to `ObjectListQuery.add_objects()` (`identifier_foo` argument) that will retrieve the identifier from each object.

`ObjectListQuery` puts no checks on whether your identifiers are actually unique.

In [9]:
@dataclass
class Specimen:
    idx: int
    name: str
    caught: datetime.datetime
    height: float
    legs: int
    arms: int

specimens_as_class_objects = [
    Specimen(idx=67, name="horse", caught=datetime.datetime(1897, 10, 10), height=1.4, legs=4, arms=0),
    Specimen(idx=180, name="cat", caught=datetime.datetime(1967, 5, 21), height=0.4, legs=4, arms=0),
    Specimen(idx=201, name="gorilla", caught=datetime.datetime(1978, 2, 10), height=0.8, legs=2, arms=2),
    Specimen(idx=-7, name="spider", caught=datetime.datetime(1942, 10, 1), height=0.03, legs=8, arms=0),
    Specimen(idx=32, name="lobster", caught=datetime.datetime(2005, 6, 12), height=0.1, legs=6, arms=2),
]

querier = ObjectListQuery(fields={ 
    ("name", "nm", str): (lambda x: x.name),
    ("caught", "ct", datetime.datetime): (lambda x: x.caught),
    ("height", "ht", float): (lambda x: x.height),
    ("legs", "lg", int): (lambda x: x.legs),
    ("arms", "ar", int): (lambda x: x.arms),
})

querier.add_objects(specimens_as_class_objects, identifier_foo=(lambda x: x.idx))

querier.query(">2[legs] AND *er[name]")

[Specimen(idx=32, name='lobster', caught=datetime.datetime(2005, 6, 12, 0, 0), height=0.1, legs=6, arms=2),
 Specimen(idx=-7, name='spider', caught=datetime.datetime(1942, 10, 1, 0, 0), height=0.03, legs=8, arms=0)]

### Performance

I haven't worked *that much* on performance, but it ain't bad. 

It appears to be O(n) on the number of objects but is somewhat sensitive to the complexity of the query.

In [10]:
# Number of objects to test with.
n_obj = 20_000

# Querier.
querier = ObjectListQuery(fields={
    ("name", "nm", str): (lambda x: x["name"]),
    ("caught", "cg", datetime.datetime): (lambda x: x["caught"]),
    ("height", "hg", float): (lambda x: x["dimensions"]["height"]),
    ("legs", "lg", int): (lambda x: x["appendages"]["legs"]),
    ("arms", "ar", int): (lambda x: x["appendages"]["arms"]),
    ("flys", "fy", bool): (lambda x: x["abilities"]["flys"]),
})

# Some animal names and caught dates to use.
names = ["duck", "rabbit", "goat", "whale", "horse", "beastie", "elk", "antelope", "gnu", "penguin"]
caught_range = [
    datetime.datetime(1900, 1, 1).timestamp(), datetime.datetime(2000, 1, 1).timestamp()
]

# Add random objects.
querier.add_objects([
    {
        "name": random.choice(names), "caught": datetime.datetime.fromtimestamp(random.uniform(*caught_range)),
        "dimensions": {"height": random.uniform(0.1, 1.5), "length": random.uniform(0.01, 20)}, 
        "appendages": {"legs": random.randint(0, 4), "arms": random.randint(0, 2)}, 
        "abilities": {"flys": random.random() > 0.5}
    }
    for _ in range(n_obj)
])

# Time for a query.
t0 = time.perf_counter()
objs = querier.query("*el*[nm] and (>0.5[hg] or >1950-10-01[caught])")
dt = 1000 * (time.perf_counter() - t0)

print(f"{len(objs)} of {n_obj} objects returned in {dt:.3f} ms")

3470 of 20000 objects returned in 20.409 ms


## SQL databases: SQLite

`SQLiteQuery(Query)` converts a boolean query into a SQLite query.

In [11]:
# Connect to an example database.
# This contains an "animal" table.
db = sqlite3.connect("animals.sqlite3")

In [12]:
querier = SQLiteQuery(fields={
    ("name", "nm", str): "name",
    ("caught", "cg", datetime.datetime): "caught",
    ("height", "hg", float): "height",
    ("legs", "lg", int): "legs",
    ("arms", "ar", int): "arms",
    ("flys", "fy", bool): "flys",
}, table="specimens")

In [13]:
query = querier.query("(<2000-01-01[caught] or z*[nm]) AND <10[LEGS]")
print(query)
list(db.execute(query))

SELECT * FROM specimens WHERE ((CAST(strftime('%s', "caught") AS INT) < 946684800 OR "name" LIKE 'z%') AND "legs" < 10)


[('zebra', '2010-03-15', 1.4, 1.8, 4, 0, 0),
 ('duck', '1987-11-03', 0.15, 0.24, 2, 0, 1),
 ('whale', '1910-01-15', 2.1, 5.6, 2, 0, 0)]

In [14]:
query = querier.query("not (t[flys] or >10[lg])")
print(query)
list(db.execute(query))

SELECT * FROM specimens WHERE NOT (("flys" = 1 OR "legs" > 10))


[('zebra', '2010-03-15', 1.4, 1.8, 4, 0, 0),
 ('monkey', '2002-04-11', 1.2, None, 2, 2, 0),
 ('whale', '1910-01-15', 2.1, 5.6, 2, 0, 0)]

In [15]:
db.close()

## NoSQL databases: MongoDB

`MongoQuery` converts a boolean query into a MongoDB query dictionary for PyMongo. The fields argument for the constructor is the same as for `ObjectListQuery`, except the function to return the value of a document/dictionary field is replaced by the period-delimited path of the field. e.g.,

`lambda x: x["a"]["b"]   =>  "a.b"`

I have nor included an actual MongoDB database here - you can do that yourself!

https://www.mongodb.com/docs/manual/tutorial/query-documents/

https://www.mongodb.com/docs/manual/tutorial/query-embedded-documents/

In [16]:
querier = MongoQuery(fields={
    ("name", "nm", str): "name",
    ("caught", "cg", datetime.datetime): "caught",
    ("height", "hg", float): "dimensions.height",
    ("legs", "lg", int): "appendages.legs",
    ("arms", "ar", int): "appendages.arms",
    ("flys", "fy", bool): "abilities.flys",
})

querier.query("*el*[nm] and (>0.5[hg] or >1950-10-01[cg])")

{'$and': [{'name': {'$regex': 'el'}},
  {'$or': [{'dimensions.height': {'$gt': 0.5}},
    {'caught': {'$gt': datetime.datetime(1950, 10, 1, 0, 0)}}]}]}